# Fracture Data Analysis Code   ~ (* *) ~

#### For this lab, your output data will consist of load vs displacement for all the samples tested. To calculate the fracture toughness of a given material, you will:
- Take the load displacement data
- Calculate the initial slope of the data as you did in the tension lab
- Construct another line with a slope equal to 95% of the initial slope
- Find the intersection of the constructed line with the original load displacement curve and record that value
- You will insert this value into equation (1) in the manual and obtain the fracture toughness of that sample

All the Best!

## Define and Import Sample Data

In [ ]:
import pandas as pd

#Define the sample name
fileName = 'Fracture Data.xlsx' #this is your data file

##INPUT ALL DIMENSIONS HERE IN METERS
W = 0.025   #Sample Width (m)
B = 0.009  #Sample thickness (m)
a = 0.0059  #Crack length (m)
sy = 40e6    #Yield Strength (Pa)

#Read the file into Data
allData = pd.read_excel(fileName, skiprows=3, sheet_name=None) #This reads all the excel sheets in
samples = list(allData.keys()) #this returns all the sheet names
print(samples)

#Choose the data you want to analyze
#YOU WILL NEED TO CHANGE THE samples[#] TO ACCESS DIFFERENT SHEETS
Data = allData[samples[9]] #This accesses individual sheets

#Save Variables to make code cleaner
Disp = Data['COD Gauge (mm)'].values
Load = Data['Load (N)'].values

## Define Some Useful Functions

In [ ]:
import numpy as np
from scipy.stats import linregress #Linear regression function built into the Scipy library

def slopeFit(Displacement,Load,a,b):
    '''This is a linear fit to data between the data indices for a and b. 
    Note, this will return an error if a or b are outside the length of 
    the load and displacement.'''
    
    #Fit the modulus
    #Linregress outputs the slope, intercept, regression, P-value and standard error
    slope,C,R,P,Err = linregress(Displacement[a:b],Load[a:b]) 
    #Note: Python lets you save multivariable outputs with a comma, i.e. a,b=[1,2] will give a=1 and b=2
    
    #Make a line for the fit data
    Y = [0.0, max(Load)] #this is a list of length 2 for plotting the fit data later
    X = [(y-C)/slope for y in Y] #these are points that you can plot to visualize later
    return slope,C,R,X,Y

def F(al):
    '''Returns the geometric stress concentration factor F'''
    return (2+al)*(0.886+4.64*al-13.32*al**2+14.72*al**3-5.6*al**4)/(1-al)**1.5 

def conditionalToughness(P,W,B,a):
    '''Returns a conditional fracture toughness'''
    return P*F(a/W)/B/W**0.5

def lineIntercepts(A,B):
    '''Returns the indexes of the points right before two curves A and B intercept'''
    intercept = np.argwhere(np.diff(np.sign(A-B))).flatten()
    if len(intercept) == 0:
        return [len(A)]
    else:
        return intercept

## Define a Data Fit and Plot

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.gca()

#YOU WILL NEED TO MODIFY THE FIT POINTS
a1 = 20; b1 = 100
slope,C,R,X,Y = slopeFit(Disp, Load, a1, b1)

#Plot the data
ax.plot(Disp, Load,'.-')
ax.plot(Disp[a1],Load[a1],'bd')
ax.plot(Disp[b1],Load[b1],'bs')

#Plot the fit
ax.plot(X,Y,label='Fit, slope='+str(round(slope,1))+' N/mm')

plt.title("Fracture Data")
ax.set_ylabel('Load (N)')
ax.set_xlabel('Displacement (mm)')
plt.legend()
plt.show()

## Determine PQ and Pmax

In [ ]:
#Find Peak Load
iPmax = np.argmax(Load)
Pmax = Load[iPmax]

#Line with 95% slope
Y95 = Load[:iPmax]
X95 = [(y-C)/slope/0.95 + 0.05*C/slope for y in Y95]

#Find intersection point and load
iPQ = lineIntercepts(Disp[:iPmax],X95)[-1]
PQ = Load[iPQ]

#Plot Data
fig = plt.figure()
ax = fig.gca()
ax.plot(Disp, Load)

#Plot fit lines
ax.plot(X,Y,'k--')
ax.plot(X95,Y95,'b--')
ax.plot(Disp[iPQ],PQ,'ro',label='PQ')
ax.plot(Disp[iPmax],Pmax,'bo',label='Pmax')

#Add Labels
ax.set_title("Intercept Fit")
ax.set_ylabel('Load (N)')
ax.set_xlabel('Displacement (mm)')
plt.legend();
plt.show()

## Determine if KQ is a valid KIC

In [ ]:
#Check Type III fracture condition
if iPQ > iPmax:
    PQ = Pmax

#Calculate the conditional toughness
KQ = conditionalToughness(PQ,W,B,a)

#Check if the conditional toughness is a true fracture toughness
#YOU WILL NEED TO DEFINE CONDITIONS 1 AND 2
if Condition1 and Condition2:
    print('KIc =',round(KQ*1e-6,3),'MPa m^0.5 is valid.')
elif Condition 1:
    print('KQ =',round(KQ*1e-6,3),'MPa m^0.5 is not valid by condition 2.')
elif Condition 2:
    print('KQ =',round(KQ*1e-6,3),'MPa m^0.5 is not valid by condition 1.')
else:
    print('KQ =',round(KQ*1e-6,3),'MPa m^0.5 and fails both conditions.')